In [95]:
GYRO = False
SEQUENCE_LENGTH = 5
SEQUENCE_OVERLAP = 4
BATCH_SIZE = 5
EPOCHS = 50
POSITION = "sitting_or_standing"
MODEL_NAME = f"position:{POSITION}_epochs:{EPOCHS}_batch:{BATCH_SIZE}_gyro:{GYRO}_window:{SEQUENCE_LENGTH}_overlap:{SEQUENCE_OVERLAP}"
DEV_SIZE = 3
TEST_SIZE = 6
LEAVE_ONE_OUT = False

### This code takes in a position, eg "sitting_or_standing", "lying_down_left" etc and trains a model for just classifying activities of that position

In [96]:
import file_tagger
import sequence_genrator
import tensorflow as tf
import numpy as np
from keras import layers, Sequential, models
import split_by_student


POSITIONS = {
    "sitting_or_standing",
    "lying_down_back",
    "lying_down_stomach",
    "lying_down_right",
    "lying_down_left"
}

STANDING_OR_SITTING_ACTIVITIES = {
    "sitting&coughing",
    "sitting&hyperventilating",
    "sitting&normal_breathing",
    
    "standing&coughing",
    "standing&hyperventilating",
    "standing&normal_breathing",
    
    "sitting&talking",
    "sitting&eating",
    "sitting&singing",
    "sitting&laughing",
    
    "standing&talking",
    "standing&eating",
    "standing&singing",
    "standing&laughing",

}

STANDING_OR_SITTING_OUTCOMES= {
    "sitting_or_standing&normal_breathing",
    "sitting_or_standing&coughing",
    "sitting_or_standing&hyperventilating",
    "sitting_or_standing&other"
    


}


LYING_DOWN_LEFT_ACTIVITIES = {
    "lying_down_left&coughing",
    "lying_down_left&hyperventilating",
    "lying_down_left&talking",
    "lying_down_left&singing",
    "lying_down_left&laughing",
    "lying_down_left&normal_breathing"

}

LYING_DOWN_LEFT_OUTCOMES = {
    "lying_down_left&normal_breathing",
    "lying_down_left&coughing",
    "lying_down_left&hyperventilating",
    "lying_down_left&other"

}



LYING_DOWN_RIGHT_ACTIVITIES = {
    "lying_down_right&normal_breathing",
    "lying_down_right&coughing",
    "lying_down_right&hyperventilating",
    "lying_down_right&talking",
    "lying_down_right&singing",
    "lying_down_right&laughing"
}

LYING_DOWN_RIGHT_OUTCOMES = {
    "lying_down_right&normal_breathing",
    "lying_down_right&coughing",
    "lying_down_right&hyperventilating",
    "lying_down_right&other"
}

LYING_DOWN_BACK_ACTIVITIES = {
    "lying_down_back&normal_breathing",
    "lying_down_back&coughing",
    "lying_down_back&hyperventilating",
    "lying_down_back&talking",
    "lying_down_back&singing",
    "lying_down_back&laughing",
}

LYING_DOWN_BACK_OUTCOMES = {
    "lying_down_back&normal_breathing",
    "lying_down_back&coughing",
    "lying_down_back&hyperventilating",
    "lying_down_back&other",
}

LYING_DOWN_STOMACH_ACTIVITIES = {
    "lying_down_stomach&normal_breathing",
    "lying_down_stomach&coughing",
    "lying_down_stomach&hyperventilating",
    "lying_down_stomach&talking",
    "lying_down_stomach&singing",
    "lying_down_stomach&laughing",
}

LYING_DOWN_STOMACH_OUTCOMES= {
    "lying_down_stomach&normal_breathing",
    "lying_down_stomach&coughing",
    "lying_down_stomach&hyperventilating",
    "lying_down_stomach&other",
}

DATA_DIRECTORY = "./all_respeck"

position_activities = {"sitting_or_standing": STANDING_OR_SITTING_ACTIVITIES,
                       "lying_down_left": LYING_DOWN_LEFT_ACTIVITIES,
                       "lying_down_right": LYING_DOWN_RIGHT_ACTIVITIES,
                       "lying_down_back": LYING_DOWN_BACK_ACTIVITIES,
                       "lying_down_stomach": LYING_DOWN_STOMACH_ACTIVITIES}

position_outcomes = {"sitting_or_standing": STANDING_OR_SITTING_OUTCOMES,
                        "lying_down_left": LYING_DOWN_LEFT_OUTCOMES,
                        "lying_down_right": LYING_DOWN_RIGHT_OUTCOMES,
                        "lying_down_back": LYING_DOWN_BACK_OUTCOMES,
                        "lying_down_stomach": LYING_DOWN_STOMACH_OUTCOMES}


POSSIBLE_ACTIVITIES = position_activities[POSITION]
POSSIBLE_OUTCOMES = position_outcomes[POSITION]
LABEL_TO_INDEX = {label: idx for idx, label in enumerate(POSSIBLE_OUTCOMES)}



In [97]:
def generate_training_data(directory, sequence_length, overlap, file_names, gyro = GYRO): # if gyro is false, only accelerometer data is used

    tagged_data = []

    # group each csv file into their respective areas
    csv_dictionary = file_tagger.tag_directory(directory)

    # iterates through each activity
    for key in POSSIBLE_ACTIVITIES:

        # iterates through each csv file for the activity 
        for csv_file in csv_dictionary[key]:
            if csv_file in file_names:
                if gyro:
                    sequences = sequence_genrator.generate_sequences_from_file_with_gyroscope(directory + "/" + csv_file, sequence_length, overlap)
                else:
                    sequences = sequence_genrator.generate_sequences_from_file_without_gyroscope(directory + "/" + csv_file, sequence_length, overlap)

                # iterate through each generated sequence
                for sequence in sequences:
                    position = key.split("&")[0]
                    activity = key.split("&")[1]

                    if activity == "talking" or activity == "singing" or activity == "laughing" or activity == "eating":
                        activity = "other"

                    if position == "standing" or position == "sitting":
                        position = "sitting_or_standing"
                        
                    tagged_data.append((position + "&" + activity, sequence))

    print ("there are " + str(len(tagged_data)) + " tagged sequences in the dataset")

    return tagged_data

In [98]:
def train_model_CNN(input_data, labels_encoded, unique_labels, epochs, batch_size, validation_data):
    if GYRO:
        width = 6
    else:
        width = 3
    # Define the CNN model for your specific input shape
    model = Sequential([
        layers.Conv1D(32, 3, activation='relu', input_shape=(SEQUENCE_LENGTH*25, width)),
        layers.MaxPooling1D(2),
        layers.Dropout(0.5),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        #layers.Dropout(0.5),
        layers.Conv1D(128, 3, activation='relu'),
        layers.MaxPooling1D(2),
        #layers.Conv1D(256, 3, activation='relu'),
        #layers.MaxPooling1D(2),
        #layers.Conv1D(512, 3, activation='relu'),
        #layers.MaxPooling1D(2),
        layers.Dropout(0.5),
        layers.Flatten(),
        #layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(len(unique_labels), activation='softmax'),

    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    if len(validation_data[0]) == 0:
        model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size)
    else:
        model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    return model

In [99]:
def create_sequence_label_lists(tagged_sequences):
    sequences = [sequence for _, sequence in tagged_sequences]
    labels = [label for label, _ in tagged_sequences]
    sequences = np.array(sequences, dtype=np.float32)
    labels_encoded = [LABEL_TO_INDEX[label] for label in labels]
    labels = np.array(labels_encoded)

    return sequences, labels
    

def create_data_sets(dev_size, test_size):

    training_files, dev_files, test_files = split_by_student.split_data(students_in_dev_set= dev_size, students_in_test_set=test_size)

    tagged_training_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=training_files)
    tagged_dev_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=dev_files)
    tagged_test_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=test_files)

    train_data, train_labels = create_sequence_label_lists(tagged_training_sequences)
    dev_data, dev_labels = create_sequence_label_lists(tagged_dev_sequences)
    test_data, test_labels = create_sequence_label_lists(tagged_test_sequences)

    #print(len(train_data), len(train_labels), len(dev_data), len(dev_labels), len(test_data), len(test_labels))

    return train_data, train_labels, dev_data, dev_labels, test_data, test_labels

In [ ]:
def leave_one_out():
    students = split_by_student.get_list_of_stutents()
    test_accuracies = []
    for test_student in students:
        print("testing student: " + str(test_student))
        print("training students: " + str([student for student in students if student != test_student]))
        
        test_files, training_files = split_by_student.get_list_of_files(test_student)

        tagged_training_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=training_files)
        tagged_test_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=test_files)

        train_data, train_labels = create_sequence_label_lists(tagged_training_sequences)
        test_data, test_labels = create_sequence_label_lists(tagged_test_sequences)

        model = train_model_CNN(train_data, train_labels, POSSIBLE_OUTCOMES, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=([], [])) #batch_size, epochs
        test_loss, test_accuracy = model.evaluate(test_data, test_labels)
        test_accuracies.append(test_accuracy)
        print("for student " + str(test_student) + " the accuracy is " + str(test_accuracy))
        print("average accuracy so far: " + str(sum(test_accuracies)/len(test_accuracies)))

        
    print("Accuracy for each student:")
    print(", ".join([f"{student}: {accuracy}" for student, accuracy in zip(students, test_accuracies)]))
    print("Average overall accuracy:", sum(test_accuracies)/len(test_accuracies))


In [100]:
if LEAVE_ONE_OUT:
    leave_one_out()
    exit()


train_data, train_labels, dev_data, dev_labels, test_data, test_labels = create_data_sets(dev_size=DEV_SIZE, test_size=TEST_SIZE)


# train and save model (CHOOSE BETWEEN CNN AND LSTM)
model = train_model_CNN(train_data, train_labels, POSSIBLE_OUTCOMES, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(dev_data, dev_labels)) #batch_size, epochs


# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Save the trained model
#model.save(f"models/models_for_presentation/lying_down_left_model.keras")

Train Set: s100, s16, s51, s80, s52, s35, s40, s61, s43, s15, s96, s79, s88, s22, s64, s36, s50, s54, s66, s11, s13, s38, s18, s102, s5, s42, s33, s3, s48, s98, s32, s21, s83, s30, s93, s97, s1, s8, s92, s71, s55, s75, s74, s77, s44, s45, s87, s67, s59, s17, s12, s29, s57, s91, s60, s39, s95, s7, s23, s70, s56, s84
Dev Set: s34, s65, s46
Test Set: s86, s82, s27, s72, s9, s63
there are 22450 tagged sequences in the dataset
there are 1086 tagged sequences in the dataset
there are 2184 tagged sequences in the dataset
Epoch 1/50
4490/4490 [==============================] - 14s 3ms/step - loss: 1.0738 - accuracy: 0.5783 - val_loss: 1.0331 - val_accuracy: 0.5847
Epoch 2/50
4490/4490 [==============================] - 14s 3ms/step - loss: 0.8809 - accuracy: 0.6018 - val_loss: 1.0350 - val_accuracy: 0.6087
Epoch 3/50
4490/4490 [==============================] - 14s 3ms/step - loss: 0.8248 - accuracy: 0.6237 - val_loss: 1.1721 - val_accuracy: 0.6041
Epoch 4/50
4490/4490 [=======================